# Tumor Landscape Analysis

Post-processing Module
---

## Barrett's Esophagus Project

Pre-processing of data 

Collaboration with **Justin Law** (<justin.law@icr.ac.uk>) and **Joe Brown** (<Joel.Brown@moffitt.org>)

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os, sys
from IPython.display import clear_output

def update_progress(progress, msg):
    bar_length = 25
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = "Progress : [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text + "  <<< " + msg)

# get parent directory
main_pth = os.path.dirname(os.getcwd())
dat_pth = main_pth + '/data/BE/'

units = '[um]'
# scale (units/pixel)
scale = 0.45


print('working directory is: ' + main_pth)

working directory is: /Volumes/GoogleDrive/My Drive/EcoEvo of Cancer/studies/BarrettsEsophagus


In [2]:
# reads classes info to dataframe
classes = pd.read_csv(main_pth + '/data/classes.csv')
classes = classes.loc[classes['drop'] == False].copy().reset_index()
classes

,index,class_code,class_word,class_name,class_val,class_color,drop
0,0,s,stromal,Stromal cells,42,#2b83ba,False
1,1,l,lymphocyte,Lymphocytes,84,#1a9641,False
2,2,t,tumor,BE cells,126,#d7191c,False
3,3,e,epithelial,Epithelial cells,168,#ff8400,False


In [3]:
#setname = 'biopsies'
setname = 'biopsies_21'
#setname = 'biopsies_all'

# reads samples result data 
sample_info = pd.read_csv(dat_pth + setname + '.csv')

outdir = dat_pth + setname + '_results/'
if not os.path.exists(outdir):
    os.makedirs(outdir)

vildir = outdir + 'violins/all_biopsies/'
if not os.path.exists(vildir):
    os.makedirs(vildir)
    
vildirpid = outdir + 'violins/pid_means/'
if not os.path.exists(vildirpid):
    os.makedirs(vildirpid)

sample_info

,sample_ID,case_ID,patient_ID,set,max_diag,code,ndpi,biopsy,atyp,image_file,coord_file,tiff_file,results_dir,row_min,row_max,col_min,col_max,row_ref,col_ref,num_points
0,D0001_1,D0001,74,Case,High-Grade,D0001_74_121T_C,3,1,NaN,NaN,biopsies/D0001_1.csv,rasters/D0001_1.tiff,results/D0001_1,6319,11954,48192,49927,6099,47972,7094
1,D0001_2,D0001,74,Case,High-Grade,D0001_74_121T_C,3,2,NaN,NaN,biopsies/D0001_2.csv,rasters/D0001_2.tiff,results/D0001_2,1153,17803,50008,53561,933,49788,16427
2,D0001_3,D0001,74,Case,High-Grade,D0001_74_121T_C,3,3,NaN,NaN,biopsies/D0001_3.csv,rasters/D0001_3.tiff,results/D0001_3,4004,13973,54122,57233,3784,53902,15171
3,D0001_4,D0001,74,Case,High-Grade,D0001_74_121T_C,3,4,NaN,NaN,biopsies/D0001_4.csv,rasters/D0001_4.tiff,results/D0001_4,4263,14856,56023,59862,4043,55803,9845
4,D0002_1,D0002,295,Case,High-Grade,D0002_295_38S_E1,3,1,NaN,NaN,biopsies/D0002_1.csv,rasters/D0002_1.tiff,results/D0002_1,8534,15246,54004,55998,8314,53784,7533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,D0036_0,D0036,938,Control,High-Grade,D0036_938_44A_B,3,0,1.0,images/D0036_0.tiff,biopsies/D0036_0.csv,rasters/D0036_0.tiff,results/D0036_0,2112,11167,10219,13394,1892,9999,8045
128,D0036_1,D0036,938,Control,High-Grade,D0036_938_44A_B,3,1,3.0,images/D0036_1.tiff,biopsies/D0036_1.csv,rasters/D0036_1.tiff,results/D0036_1,7922,11975,7431,9153,7702,7211,6734
129,D0036_2,D0036,938,Control,High-Grade,D0036_938_44A_B,3,2,3.0,images/D0036_2.tiff,biopsies/D0036_2.csv,rasters/D0036_2.tiff,results/D0036_2,994,11965,221,2892,774,1,21094
130,D0036_3,D0036,938,Control,High-Grade,D0036_938_44A_B,3,3,3.0,images/D0036_3.tiff,biopsies/D0036_3.csv,rasters/D0036_3.tiff,results/D0036_3,4028,8238,7712,9053,3808,7492,5341


In [4]:
stats = pd.DataFrame()
MH_metrics_tbl = pd.DataFrame()
LME_fracs = pd.DataFrame()

for index, sample in sample_info.iterrows():
    
    sid = sample.sample_ID
    cid = sample.case_ID
    diag = sample.max_diag
    pid = sample.patient_ID
    st = sample.set
    msg = sid + " : [{0}/{1}]".format( index + 1, len(sample_info.index))
    resdir = dat_pth + 'results/' + sid
    
    update_progress(index/len(sample_info), msg)
    
    result_dir = sample.results_dir
    
    fil = resdir + '/' + sid +'_'+ setname + '_result_stats_lme.tsv'
    if os.path.exists(fil):
        aux = pd.read_csv(fil, sep='\t')
        
        aux.insert(loc=1, column='pid',      value=pid)
        aux.insert(loc=2, column='set',      value=st)        
        aux.insert(loc=3, column='max_diag', value=diag)
        stats = stats.append(aux, ignore_index=True)
    
    fil = resdir + '/LME_sample_metrics_'+ setname + '.tsv'
    if os.path.exists(fil):
        aux = pd.read_csv(fil, sep='\t')
        
        aux.insert(loc=0, column='sample_ID', value=sid)
        aux.insert(loc=1, column='pid',       value=pid)
        aux.insert(loc=2, column='set',       value=st)
        aux.insert(loc=3, column='max_diag',  value=diag)
        MH_metrics_tbl = MH_metrics_tbl.append(aux, ignore_index=True)

        
    fil = resdir + '/patch_level/LME_patch_metrics_'+ setname + '.tsv'
    if os.path.exists(fil):
        aux = pd.read_csv(fil, sep='\t')
        
        auy = aux[['LME', 'area_fraction']].groupby(['LME']).sum().reset_index()
        auy.insert(loc=0, column='sample_ID', value=sid)
        auy.insert(loc=1, column='pid',       value=pid)
        auy.insert(loc=2, column='set',       value=st)
        auy.insert(loc=3, column='max_diag',  value=diag)
        LME_fracs = LME_fracs.append(auy, ignore_index=True) 
        
    
update_progress(1, msg)

Progress : [#########################] 100.0%  <<< D0036_4 : [132/132]


In [5]:
LME_fracs.to_csv(outdir + setname + '_result_lme_fractions.tsv', sep='\t', index=False)

stats['t2s'] = np.nan
stats.loc[stats['s_density'] > 0, 't2s'] = stats['t_density']/stats['s_density']

stats.to_csv(outdir + setname + '_result_stats.tsv', sep='\t', index=False)
stats


,sample_ID,pid,set,max_diag,code,ndpi,biopsy,atyp,num_points,total_area,...,LME_shape_index,LME_dominance,LME_num_patches,LME_patch_density,LME_total_edge,LME_edge_density,LME_largest_patch_index,e_fraction,e_density,t2s
0,D0001_1,74,Case,High-Grade,D0001_74_121T_C,3,1,NaN,7094,14062500.0,...,4.000000,0.217893,22.0,22.222222,98.0,0.989899,20.202020,NaN,NaN,0.396381
1,D0001_2,74,Case,High-Grade,D0001_74_121T_C,3,2,NaN,16427,73312500.0,...,5.689655,0.235380,53.0,26.108374,174.0,0.857143,21.182266,NaN,NaN,0.944816
2,D0001_3,74,Case,High-Grade,D0001_74_121T_C,3,3,NaN,15171,39375000.0,...,4.461538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.513376
3,D0001_4,74,Case,High-Grade,D0001_74_121T_C,3,4,NaN,9845,50625000.0,...,4.729167,0.275278,42.0,31.343284,115.0,0.858209,19.402985,NaN,NaN,1.139312
4,D0002_1,295,Case,High-Grade,D0002_295_38S_E1,3,1,NaN,7533,18125000.0,...,3.428571,0.487740,17.0,15.454545,64.0,0.581818,65.454545,NaN,NaN,0.771369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,D0036_0,938,Control,High-Grade,D0036_938_44A_B,3,0,1.0,8045,36562500.0,...,5.375000,0.504514,32.0,17.391304,103.0,0.559783,44.021739,NaN,NaN,0.201957
128,D0036_1,938,Control,High-Grade,D0036_938_44A_B,3,1,3.0,6734,10687500.0,...,3.605263,0.229489,24.0,28.571429,87.0,1.035714,21.428571,NaN,NaN,1.417706
129,D0036_2,938,Control,High-Grade,D0036_938_44A_B,3,2,3.0,21094,37375000.0,...,6.333333,0.256947,58.0,21.886792,278.0,1.049057,18.490566,NaN,NaN,1.185112
130,D0036_3,938,Control,High-Grade,D0036_938_44A_B,3,3,3.0,5341,9500000.0,...,3.852941,0.187980,24.0,34.782609,79.0,1.144928,17.391304,1.104662,0.001306,0.519454


In [6]:
import seaborn as sns
from statannot import add_stat_annotation
from itertools import combinations
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

def violin_strat(dat, classes, signal, ttl, fname):
    
    orig_stdout = sys.stdout
    f = open(fname +'.txt', 'w')
    sys.stdout = f
    
    aux = dat[classes].to_list()
    auy = dat[signal].to_list()
    cls = sorted(set(aux), reverse=True)
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 12), facecolor='w', edgecolor='k')
    
    sns.axes_style("whitegrid")
    sns.violinplot(x  = aux, 
                   y  = auy, 
                   ax = ax,
                   palette ="Set3", 
                   scale = 'count', 
                   inner = 'box',
                   order = cls)
    sns.swarmplot(x  = aux, 
                  y  = auy, 
                  ax = ax,
                  color = "black",
                  order = cls)
    
    add_stat_annotation(ax, x=aux, y=auy, order=cls,
                        box_pairs=combinations(cls, 2),
                        test='t-test_ind', 
                        #text_format='full',
                        text_format='star', 
                        loc='inside', verbose=2);    
    ax.set_title(ttl)
    
    plt.savefig(fname +'.png', bbox_inches='tight', dpi=90)
    plt.close()
    sys.stdout = orig_stdout
    f.close()
         
    return([fig, ax])


def violin_groups(dat, classes, signal, groups, ttl, fname):
    
    orig_stdout = sys.stdout
    f = open(fname +'.txt', 'w')
    sys.stdout = f
    
    aux = dat[groups].to_list()
    grps = sorted(set(aux), reverse=False)
    comb = list(combinations(grps, 2))
    
    aux = dat[classes].to_list()
    cls = sorted(set(aux), reverse=False)
    
    box_pairs=[]
    for cas in comb:
        box_pairs += [((x, cas[0]),(x, cas[1])) for x in cls]
    

    fig, ax = plt.subplots(1, 1, figsize=(12, 12), facecolor='w', edgecolor='k')
    
    sns.axes_style("whitegrid")
    sns.violinplot(data= dat,
                   x  = classes, 
                   y  = signal, 
                   hue = groups,
                   ax = ax,
                   palette ="Set3", 
                   #scale = 'count', 
                   scale = 'width', 
                   inner = 'box',
                   order = cls)
    handles = ax.legend_.legendHandles
    labels = [text.get_text() for text in ax.legend_.texts]
    
    sns.swarmplot(data= dat,
                  x  = classes, 
                  y  = signal,
                  hue = groups,
                  dodge=True,
                  ax = ax,
                  color = "black",
                  order = cls,
                  size = 3)
     
    add_stat_annotation(ax, data= dat, 
                        x=classes, y=signal, hue = groups, 
                        order=cls,
                        box_pairs=box_pairs,
                        test='t-test_ind', 
                        #text_format='full',
                        text_format='star', 
                        loc='inside', verbose=2);    
    ax.set_title(ttl)
    ax.set_xlabel('LME')
    ax.set_ylabel('Area fraction')
    ax.legend(handles, labels, title='', bbox_to_anchor=(1.01, 1), loc='upper left')
     
    plt.savefig(fname +'.png', bbox_inches='tight', dpi=90)
    plt.close()
    sys.stdout = orig_stdout
    f.close()
         
    return([fig, ax])

noncols = ['sample_ID', 'pid', 'set', 'max_diag', 'code', 
           'ndpi', 'biopsy', 'atyp', 'p_value',
           'num_points', 'total_area', 'ROI_area', 'num_cells']


In [7]:
# Plot violin distributions for LME fractions in all biopsies and patients
# grouped by LME and set (control vs case) 
fig, ax = violin_groups(LME_fracs, 'LME', 'area_fraction', 'set', 'LME fractions',
                        vildir + 'lme_fracs')

In [8]:
# Plot violin distributions for simple statistics, for all biopsies and patients
# grouped by set (control vs case) 
cols = [x for x in stats.columns if x not in noncols]
for col in cols:
    fig, ax = violin_strat(stats, 'set', col, col, vildir + 'stat_' + col)
    
aux = stats[cols + ['set', 'pid']].groupby(['pid', 'set']).mean().reset_index().drop(['pid'], axis=1)
for col in cols:
    fig, ax = violin_strat(aux, 'set', col, col, vildirpid + 'stat_pid_' + col)
    

In [9]:
# Plot violin distributions for MH statistics, for all biopsies and patients
# grouped by set (control vs case)
cols = [x for x in MH_metrics_tbl.columns if x not in (noncols + 
                                             ['number_of_patches', 'total_edge', 'effective_mesh_size',
                                              'fractal_dimension_mn', 'fractal_dimension_am', 
                                              'fractal_dimension_md', 'fractal_dimension_ra', 
                                              'fractal_dimension_sd', 'fractal_dimension_cv'])]
for col in cols:
    fig, ax = violin_strat(MH_metrics_tbl, 'set', col, col, vildir + 'MH_metrics_' + col)
    
cols = [x for x in MH_metrics_tbl.columns if x not in (noncols + 
                                             ['number_of_patches', 'total_edge', 'effective_mesh_size',
                                              'area_am', 'area_md', 'area_ra', 
                                              'area_sd', 'area_cv', 
                                              'perimeter_am', 'perimeter_md', 'perimeter_ra', 
                                              'perimeter_sd', 'perimeter_cv',
                                              'perimeter_area_ratio_am', 'perimeter_area_ratio_md', 
                                              'perimeter_area_ratio_ra', 'perimeter_area_ratio_sd', 
                                              'perimeter_area_ratio_cv',
                                              'shape_index_am', 'shape_index_md', 'shape_index_ra', 
                                              'shape_index_sd', 'shape_index_cv',
                                              'fractal_dimension_mn', 'fractal_dimension_am', 
                                              'fractal_dimension_md', 'fractal_dimension_ra', 
                                              'fractal_dimension_sd', 'fractal_dimension_cv',
                                              'euclidean_nearest_neighbor_am', 'euclidean_nearest_neighbor_md',
                                              'euclidean_nearest_neighbor_ra', 'euclidean_nearest_neighbor_sd',
                                              'euclidean_nearest_neighbor_cv'])]
aux = MH_metrics_tbl[cols + ['set', 'pid']].groupby(['pid', 'set']).mean().reset_index().drop(['pid'], axis=1)
for col in cols:
    fig, ax = violin_strat(aux, 'set', col, col, vildirpid + 'MH_metrics_pid_' + col)    

# Cox Regression

In [25]:
import warnings
from lifelines import CoxPHFitter
cph = CoxPHFitter()

tbl_a = pd.DataFrame()
tbl_b = pd.DataFrame()

def signif(x):
    if (x < 0.0001):
        return '****'
    elif (x < 0.001):
        return '***'
    elif (x < 0.01):
        return '**'
    elif (x < 0.05):
        return '*'
    elif (x < 0.01):
        return '.'
    else:
        return 'ns'

In [26]:
# reads samples result data 
stats = pd.read_csv(outdir + setname + '_result_stats.csv')
stats['event'] = (1*(stats['set']=='Case'))

noncols = ['sample_ID', 'pid', 'case_ID', 'ndpi', 'Age', 'set', 'max_diag', 'code', 
           'biopsy', 'atyp', 'num_points', 'total_area', 'ROI_area', 'num_cells',
          'e_fraction', 'e_density']

cols = [x for x in stats.columns if x not in (noncols + 
                                             ['Years', 'event'])]

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for col in cols:

        aux = stats[['case_ID', 'pid', 'Years', 'event', col]].dropna()
        aux = aux.loc[aux['Years'] > 0]
        grp = aux.groupby(['case_ID', 'pid', 'Years', 'event'], as_index=True).agg({col: ['mean', 'std']})
        grp.columns = ["_".join(x) for x in grp.columns.ravel()]
        grp = grp.reset_index()
        
        a = grp.sort_values(by=['Years'], ascending=False)
        a = a.drop_duplicates(subset=['pid']).reset_index(drop=True)
        cph.fit(a.drop(['case_ID', 'pid'], axis=1), duration_col='Years', event_col='event')
        tbl_i = cph.summary.reset_index()
        tbl_i['significance'] = tbl_i.apply(lambda row : signif(row['p']), axis = 1)
        tbl_a = tbl_a.append(tbl_i, ignore_index=True)
        
        
        b = grp.sort_values(by=['Years'], ascending=True)
        b = b.drop_duplicates(subset=['pid']).reset_index(drop=True)
        cph.fit(b.drop(['case_ID', 'pid'], axis=1), duration_col='Years', event_col='event')
        tbl_i = cph.summary.reset_index()
        tbl_i['significance'] = tbl_i.apply(lambda row : signif(row['p']), axis = 1)
        tbl_b = tbl_b.append(tbl_i, ignore_index=True)

#tbl_a.to_csv(vildir + 'stats_cox_regs_longest_time.tsv',  sep='\t', index=False)
#tbl_b.to_csv(vildir + 'stats_cox_regs_shortest_time.tsv', sep='\t', index=False)


In [27]:
metrics_tbl = pd.merge(MH_metrics_tbl, 
               stats[['sample_ID', 'case_ID', 'Years', 'event']], 
               on='sample_ID')

noncols = ['sample_ID', 'pid', 'case_ID', 'ndpi', 'Age', 'set', 'max_diag',  
           'total_area''number_of_patches', 'total_edge', 'effective_mesh_size',
           'area_am', 'area_md', 'area_ra','area_sd', 'area_cv',
           'perimeter_am', 'perimeter_md', 'perimeter_ra', 'perimeter_sd', 'perimeter_cv',
           'perimeter_area_ratio_am', 'perimeter_area_ratio_md',
           'perimeter_area_ratio_ra', 'perimeter_area_ratio_sd','perimeter_area_ratio_cv',
           'shape_index_am', 'shape_index_md', 'shape_index_ra', 'shape_index_sd', 'shape_index_cv',
           'fractal_dimension_mn', 'fractal_dimension_am', 'fractal_dimension_md', 'fractal_dimension_ra', 
           'fractal_dimension_sd', 'fractal_dimension_cv',
           'euclidean_nearest_neighbor_am', 'euclidean_nearest_neighbor_md',
           'euclidean_nearest_neighbor_ra', 'euclidean_nearest_neighbor_sd',
           'euclidean_nearest_neighbor_cv']

cols = [x for x in metrics_tbl.columns if x not in (noncols + 
                                             ['Years', 'event'])]

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for col in cols:

        aux = metrics_tbl[['case_ID', 'pid', 'Years', 'event', col]].dropna()
        aux = aux.loc[aux['Years'] > 0]
        grp = aux.groupby(['case_ID', 'pid', 'Years', 'event'], as_index=True).agg({col: ['mean', 'std']})
        grp.columns = ["_".join(x) for x in grp.columns.ravel()]
        grp = grp.reset_index()
        
        a = grp.sort_values(by=['Years'], ascending=False)
        a = a.drop_duplicates(subset=['pid']).reset_index(drop=True)
        cph.fit(a.drop(['case_ID', 'pid'], axis=1), duration_col='Years', event_col='event')
        tbl_i = cph.summary.reset_index()
        tbl_i['significance'] = tbl_i.apply(lambda row : signif(row['p']), axis = 1)
        tbl_a = tbl_a.append(tbl_i, ignore_index=True)
        
        b = grp.sort_values(by=['Years'], ascending=True)
        b = b.drop_duplicates(subset=['pid']).reset_index(drop=True)
        cph.fit(b.drop(['case_ID', 'pid'], axis=1), duration_col='Years', event_col='event')
        tbl_i = cph.summary.reset_index()
        tbl_i['significance'] = tbl_i.apply(lambda row : signif(row['p']), axis = 1)
        tbl_b = tbl_b.append(tbl_i, ignore_index=True)


tbl_a = tbl_a.sort_values(by=['p'], ascending=True)
tbl_b = tbl_b.sort_values(by=['p'], ascending=True)
tbl_a.to_csv(vildir + 'stats_cox_regs_longest_time.tsv',  sep='\t', index=False)
tbl_b.to_csv(vildir + 'stats_cox_regs_shortest_time.tsv', sep='\t', index=False)


In [24]:
tbl_a

,covariate,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p),significance
12,t_density_mean,109.335028,3.045086e+47,46.646004,17.910541,200.759516,6.004118e+07,1.544364e+87,2.343931,0.019082,5.711668,*
10,t_fraction_mean,0.211873,1.235991e+00,0.103077,0.009846,0.413900,1.009895e+00,1.512706e+00,2.055484,0.039832,4.649918,*
3,s_fraction_std,0.141839,1.152391e+00,0.069801,0.005032,0.278646,1.005045e+00,1.321340e+00,2.032056,0.042148,4.568393,*
56,perimeter_area_ratio_mn_mean,-0.001215,9.987854e-01,0.000691,-0.002570,0.000140,9.974330e-01,1.000140e+00,-1.757963,0.078754,3.666507,ns
26,LME_dominance_mean,7.336023,1.534597e+03,5.171746,-2.800413,17.472459,6.078493e-02,3.874294e+07,1.418481,0.156050,2.679916,ns
...,...,...,...,...,...,...,...,...,...,...,...,...
55,perimeter_mn_std,-0.064388,9.376415e-01,0.440705,-0.928153,0.799378,3.952831e-01,2.224157e+00,-0.146101,0.883841,0.178141,ns
19,LME_adjacency_index_std,1.311966,3.713466e+00,10.939082,-20.128240,22.752171,1.813077e-09,7.605758e+09,0.119934,0.904536,0.144751,ns
49,edge_density_std,-0.822690,4.392484e-01,7.766924,-16.045581,14.400200,1.075209e-07,1.794434e+06,-0.105922,0.915644,0.127141,ns
28,LME_num_patches_mean,0.002129,1.002131e+00,0.028016,-0.052782,0.057040,9.485867e-01,1.058698e+00,0.075994,0.939424,0.090151,ns


# SPECIFIC PLOTS

In [ ]:
import seaborn as sns
from statannot import add_stat_annotation
from itertools import combinations
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)


def violins(dat, classes, signal, lab):
    
    aux = dat[classes].to_list()
    auy = dat[signal].to_list()
    cls = sorted(set(aux), reverse=True)
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 12), facecolor='w', edgecolor='k')
    
    sns.axes_style("whitegrid")
    sns.violinplot(x  = aux, 
                   y  = auy, 
                   ax = ax,
                   palette ="Set3", 
                   scale = 'count', 
                   inner = 'box',
                   order = cls)
    sns.swarmplot(x  = aux, 
                  y  = auy, 
                  ax = ax,
                  color = "black",
                  order = cls)
    
    add_stat_annotation(ax, x=aux, y=auy, order=cls,
                        box_pairs=combinations(cls, 2),
                        test='t-test_ind', 
                        line_offset_to_box=0.2,
                        text_format='full',
                        #text_format='star', 
                        loc='inside', verbose=2);    
    ax.set_ylabel(lab)
    sns.set(font_scale = 4)
         
    return([fig, ax])

In [ ]:
cols = ['cell_density', 't_density', 't_fraction', 'l_fraction', 'LME_patch_density']
labs = [r'Cell Density $[\mu m^{-2}]$', 
        r'BE-cell Density $[\mu m^{-2}]$',
        'BE-cell Fraction',
        'Lymphocyte Fraction',  
        'Patch Density']

aux = stats[['sample_ID', 'set'] + cols].copy()
aux['cell_density'] = aux['cell_density']/(scale*scale)
aux['t_density'] = aux['t_density']/(100*scale*scale)

In [ ]:
for i, col in enumerate(cols):
    
    #i = 4
    col = cols[i]
    lab = labs[i]
    
    fig, ax = violins(aux, 'set', col, lab)
    plt.savefig(vildir + 'aaa_stat_' + col +'.png', bbox_inches='tight', dpi=90)
    plt.close()
    


In [ ]:
cols = ['shape_index_mn', 'area_mn', 'perimeter_mn']
labs = ['Mean Shape Index', 
        r'Mean Area $[\mu m^{2}]$',
        r'Mean Perimeter $[\mu m]$']

aux = MH_metrics_tbl[['sample_ID', 'set'] + cols].copy()
aux['shape_index_mn'] = 1/aux['shape_index_mn']
aux['area_mn'] = aux['area_mn']*(scale*scale)
aux['perimeter_mn'] = aux['perimeter_mn']*(scale)

In [ ]:
for i, col in enumerate(cols):
    
    #i = 4
    col = cols[i]
    lab = labs[i]
    
    fig, ax = violins(aux, 'set', col, lab)
    plt.savefig(vildir + 'aaa_stat_' + col +'.png', bbox_inches='tight', dpi=90)
    plt.close()
  